# Table Scraping Seasons and Episodes of Each Series
## Import Libraries and URL List

In [17]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
url_list = pd.read_csv('./data/series_urls.csv')

## Print seasons from each series.

In [ ]:
for index, row in url_list.iterrows():
    series = row['Series']
    # print(series)
    soup = BeautifulSoup(urlopen(row['URL']))
    table = soup.find(class_='wikitable')
    rows = table.find_all('tr')
    for row in rows[2:]:
        season_number = row.find('th').text
        tds = row.find_all('td')
        premiere_date = tds[1].span.text.strip()
        premiere_date = premiere_date[1:-1] # Removes parentheses around date.
        
        finale_date = tds[2].span
        
        # Check if the season has finished airing.
        if hasattr(finale_date, 'text'):
            finale_date = finale_date.text.strip()
            finale_date = finale_date[1:-1]
        else:
            finale_date = 'NULL'
        # print(f"Season {season_number} premiered on {premiere_date} and ended on {finale_date}")
        print(
f"""(
    '{series}',
    {season_number},
    '{premiere_date}',
    '{finale_date}'
),""")

## Print out episodes from each season.

In [ ]:
for index, row in url_list.iterrows():
    series = row['Series']
    # print(series)
    soup = BeautifulSoup(urlopen(row['URL']))
    tables = soup.find_all(class_='wikiepisodetable')
    for table in tables:
        season_number = table.caption.text
        season_number = season_number.replace(' episodes', '')
        season_number = season_number[-2:].strip()
        # print(f"Season {season_number}")
        rows = table.find_all('tr', class_='vevent')
        for row in rows:
            tds = row.find_all('td')
            episode_number = tds[0].text
            title = tds[1].text[1:-1] # Indexes exclude first and last character of string
            title = title.replace("'", "''") # Use double apostrophes or error would occur in SQL inserts
            date = tds[2].span.text.strip()
            date = date[1:-1]
            # print(f"Episode {episode_number}: {title}, {date}")
            print(
f"""(
    (
        SELECT
            season_id
        FROM
            seasons
        WHERE
            series_id = '{series}'
            AND season_number = {season_number}
    ),
    {episode_number},
    '{title}',
    '{date}'
),""")